In [ ]:
#importing libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,  GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
#loading datasets
data1 = pd.read_csv('crop_production.csv')
data2 = pd.read_csv('datafile (2).csv')
data3 = pd.read_csv('datafile (3).csv')
data4 = pd.read_csv('produce.csv')
data5 = pd.read_csv('datafile (1).csv')


In [ ]:
#function to find similar columns to merge the datasets
def find_similar_columns(df1, df2):
    similar_columns = []
    for col1 in df1.columns:
        for col2 in df2.columns:
            if col1 == col2:
                similar_columns.append(col1)
    return similar_columns

In [ ]:
similar_columns_1_2 = find_similar_columns(data1, data2)
similar_columns_1_3 = find_similar_columns(data1, data3)
similar_columns_1_4 = find_similar_columns(data1, data4)
similar_columns_1_5 = find_similar_columns(data1, data5)

In [ ]:
print("Similar columns between crop_production and datafile (2):", similar_columns_1_2)
print("Similar columns between crop_production and datafile (3):", similar_columns_1_3)
print("Similar columns between crop_production and produce.csv:", similar_columns_1_4)
print("Similar columns between crop_production and datafile .csv:", similar_columns_1_5)

Similar columns between crop_production and datafile (2): ['Crop']
Similar columns between crop_production and datafile (3): ['Crop']
Similar columns between crop_production and produce.csv: []
Similar columns between crop_production and datafile .csv: ['Crop', 'State']


In [ ]:
# Convert crop names to lowercase in data1, data3, data5, and data6
data3['Crop'] = data3['Crop'].str.lower()
data5['Crop'] = data5['Crop'].str.lower()
data1['Crop'] = data1['Crop'].str.lower()
data2['Crop'] = data2['Crop'].str.lower()


# Merge data1 with data3 on 'Crop' column
merged_data = pd.merge(data1, data3, on='Crop', how='inner')

# Merge merged_data with data5 on 'Crop' column
merged_data = pd.merge(merged_data, data5, on='Crop', how='inner')

# Merge merged_data with data2 on 'Crop' column
merged_data = pd.merge(merged_data, data2, on='Crop', how='inner')



In [ ]:
merged_data.to_csv('merged_dataset.csv', index=False)

In [ ]:
merged_data.fillna(0, inplace=True)

In [ ]:
merged_data.isnull().sum()

Crop                                     0
District_Name                            0
Crop_Year                                0
Season                                   0
State_x                                  0
Area                                     0
Production                               0
Variety                                  0
Season/ duration in days                 0
Recommended Zone                         0
State_y                                  0
Cost of Cultivation (`/Hectare) A2+FL    0
Cost of Cultivation (`/Hectare) C2       0
Cost of Production (`/Quintal) C2        0
Yield (Quintal/ Hectare)                 0
Production 2006-07                       0
Production 2007-08                       0
Production 2008-09                       0
Production 2009-10                       0
Production 2010-11                       0
Area 2006-07                             0
Area 2007-08                             0
Area 2008-09                             0
Area 2009-1

In [ ]:
read = pd.read_csv('merged_dataset.csv')

<ipython-input-90-2222dcb1b463>:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  read = pd.read_csv('merged_dataset.csv')


In [ ]:
read.shape

(865527, 32)

In [ ]:
'Yield (Quintal/ Hectare) ' in merged_data.columns

True

In [ ]:
merged_data['Total_Cost_of_Cultivation'] = merged_data['Cost of Cultivation (`/Hectare) A2+FL'] + \
                                            merged_data['Cost of Cultivation (`/Hectare) C2']
merged_data['Total_Cost_of_Production'] = merged_data['Cost of Production (`/Quintal) C2'] * \
                                           merged_data['Yield (Quintal/ Hectare) ']


In [ ]:
X = merged_data[['Cost of Cultivation (`/Hectare) A2+FL', 'Cost of Cultivation (`/Hectare) C2',
                 'Cost of Production (`/Quintal) C2', 'Yield (Quintal/ Hectare) ',
                 'Total_Cost_of_Cultivation', 'Total_Cost_of_Production']]
y = merged_data['Production']

In [ ]:
X.head()

,Cost of Cultivation (`/Hectare) A2+FL,Cost of Cultivation (`/Hectare) C2,Cost of Production (`/Quintal) C2,Yield (Quintal/ Hectare),Total_Cost_of_Cultivation,Total_Cost_of_Production
0,24538.32,45239.51,93.64,448.89,69777.83,42034.0596
1,55655.44,86765.77,86.53,986.21,142421.21,85336.7513
2,56621.16,91442.63,119.72,757.92,148063.79,90738.1824
3,57673.60,85801.95,107.56,744.01,143475.55,80025.7156
4,66335.06,89025.27,85.79,1015.45,155360.33,87115.4555


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


In [ ]:
# Load the merged dataset
merged_data = pd.read_csv('merged_dataset.csv')


<ipython-input-112-b2a7d6340837>:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.read_csv('merged_dataset.csv')


In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
y_pred = model.predict(X_test)


In [ ]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 551658847565.1456


In [ ]:
# Overfitting using Cross-Validation
# Define the model
model_cv = RandomForestRegressor(n_estimators=100, random_state=42)


In [ ]:
# Perform cross-validation
cv_scores = cross_val_score(model_cv, X, y, cv=5, scoring='neg_mean_squared_error')

In [ ]:
cv_mse = -cv_scores.mean()
print("Mean Squared Error (Cross-validation):", cv_mse if cv_mse > 0 else 0)

In [ ]:
#Hyperparameter
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)
grid_search.fit(X_train, y_train)


In [ ]:
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print("Best Parameters:", best_params)
print("Best Estimator:", best_estimator)


In [ ]:
y_pred = best_estimator.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Mean Squared Error (After Hyperparameter Tuning):", mse)
print("Mean Absolute Error (After Hyperparameter Tuning):", mae)